## SwiftUI에서 위도경도 여러개를 JSON파일로 찍어보기위해 전처리하기

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Data/제주도식당_위도경도컬럼.csv")

In [3]:
df.head()

,lat,lng
0,33.5151128,126.5202001
1,33.5214225,126.8588743
2,33.4192601,126.2674304
3,33.4714278,126.5457848
4,33.4879976,126.4978933


### csv to JSON

In [4]:
df.to_json('Data/제주도식당_위도경도JSON.json', orient='records', force_ascii=False)

---
### Map에 쓸 테스트 JSON 생성
- 테스트 데이터이기 때문에 용량을 줄일 목적으로 리뷰가 없는 데이터는 제외

In [43]:
jejuJSON = pd.read_json("Data/제주도식당Data.json")

In [45]:
jejuJSON = jejuJSON[jejuJSON['review'] != 'x']

In [46]:
jejuJSON.count(axis=1)

1       13
2       13
3       13
4       13
5       13
        ..
9991    13
9992    13
9993    13
9995    13
9998    13
Length: 5792, dtype: int64

In [48]:
jejuJSON = jejuJSON.iloc[0:,0:12]

In [50]:
jejuJSON.to_json("Data/제주도식당Data_Test.json", orient='records', force_ascii=False)

---
### 제주도 공중화장실 데이터 전처리

In [51]:
jejuToilet = pd.read_excel("Data/제주공중화장실Data_원본.xlsx")

/Users/jeongtaeyeong/miniforge3/envs/tensorflow/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [52]:
jejuToilet.head()

,번호,구분,근거,화장실명,소재지도로명주소,소재지지번주소,남성용-대변기수,남성용-소변기수,남성용-장애인용대변기수,남성용-장애인용소변기수,...,화장실소유구분,오물처리방식,안전관리시설설치대상여부,비상벨설치여부,비상벨설치장소,화장실입구CCTV설치유무,기저귀교환대유무,기저귀교환대장소,리모델링연월,데이터기준일자
0,1,개방화장실,법제3조제1호,솔오름전망대,NaN,제주특별자치도 서귀포시 동홍동 2149,1,1,0,0,...,공공기관-지방자치단체,수거식,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20
1,2,개방화장실,법제3조제1호,고근산,NaN,제주특별자치도 서귀포시 서호동 1286-1,1,1,0,0,...,공공기관-지방자치단체,수거식,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20
2,3,개방화장실,법제3조제1호,엉또폭포,제주특별자치도 서귀포시 엉또로 104,제주특별자치도 서귀포시 강정동 1561-1,1,1,0,0,...,공공기관-지방자치단체,수거식,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20
3,4,개방화장실,법제3조제1호,악근천(풍림콘도 -동쪽 바로 옆),NaN,제주특별자치도 서귀포시 강정동 5628 (신축),1,1,0,0,...,공공기관-지방자치단체,수거식,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20
4,5,개방화장실,법제3조제1호,베릿내,제주특별자치도 서귀포시 중문관광로 181,제주특별자치도 서귀포시 중문동 2631,1,1,0,0,...,공공기관-지방자치단체,수거식,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20


### 컬럼 확인
- 필요한 컬럼만 뽑기위해 필요함

In [53]:
jejuToilet.columns

Index(['번호', '구분', '근거', '화장실명', '소재지도로명주소', '소재지지번주소', '남성용-대변기수', '남성용-소변기수',
       '남성용-장애인용대변기수', '남성용-장애인용소변기수', '남성용-어린이용대변기수', '남성용-어린이용소변기수',
       '여성용-대변기수', '여성용-장애인용대변기수', '여성용-어린이용대변기수', '관리기관명', '전화번호', '개방시간',
       '개방시간상세', '설치연월', 'WGS84위도', 'WGS84경도', '화장실소유구분', '오물처리방식',
       '안전관리시설설치대상여부', '비상벨설치여부', '비상벨설치장소', '화장실입구CCTV설치유무', '기저귀교환대유무',
       '기저귀교환대장소', '리모델링연월', '데이터기준일자'],
      dtype='object')

### 제일 중요한 위도경도 값에 대한 결측치 확인

In [55]:
jejuToilet[['WGS84위도', 'WGS84경도']].isna().sum()

WGS84위도    427
WGS84경도    427
dtype: int64

> 전체 데이터가 777개인데 427개가 na값이라 도로명주소로 위도경도를 채우기로 결정

In [65]:
# NA값 존재시 API못돌려서 x로 대체
jejuToilet['소재지도로명주소'].fillna('x', inplace=True)

In [56]:
import urllib.request # network 요청 보내기
import json
import time
import random

# 네이버 API 키 정보들
client_id = ""
client_secret = ""

In [62]:
# 위도경도를 뽑아올 함수 정의
def addressToLatLng(address):
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" + urllib.parse.quote(address)
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    lat_lng = None
    if rescode == 200:
        response_body = response.read()
        entext = response_body.decode('utf-8')
        data = json.loads(entext)
        try:
            lat_lng = {
                "lat": data['addresses'][0]['y'],
                "lng": data['addresses'][0]['x']
            }
        except:
            # 주소에서 가져올 수 없을 때 x추가
            lat_lng = {"lat": "x", "lng": "x"}
    else:
        print("Error Code:" + rescode)
    return lat_lng
# 데이터프레임 초기화
df = pd.DataFrame(columns=['latitude', 'longitude'])

In [66]:
# Test
# addressToLatLng("제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)")
count = 1
# 주소별 위도와 경도를 추출하여 데이터프레임에 추가
for address in jejuToilet['소재지도로명주소']:
    lat_lng = addressToLatLng(address)
    if lat_lng:
        df = pd.concat([df, pd.DataFrame([lat_lng])], ignore_index=True)        
        print(count)
        count += 1

print("완료")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [68]:
df = df[['lat', 'lng']]

In [69]:
df

,lat,lng
0,x,x
1,x,x
2,33.2672443,126.4983941
3,x,x
4,33.2447132,126.4195338
...,...,...
772,33.4863833,126.4569079
773,33.4866347,126.4375090
774,33.4750610,126.5155220
775,33.5096823,126.7156478


In [71]:
newJejuToilet = pd.concat([jejuToilet, df], axis=1)

In [72]:
newJejuToilet.head()

,번호,구분,근거,화장실명,소재지도로명주소,소재지지번주소,남성용-대변기수,남성용-소변기수,남성용-장애인용대변기수,남성용-장애인용소변기수,...,안전관리시설설치대상여부,비상벨설치여부,비상벨설치장소,화장실입구CCTV설치유무,기저귀교환대유무,기저귀교환대장소,리모델링연월,데이터기준일자,lat,lng
0,1,개방화장실,법제3조제1호,솔오름전망대,x,제주특별자치도 서귀포시 동홍동 2149,1,1,0,0,...,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20,x,x
1,2,개방화장실,법제3조제1호,고근산,x,제주특별자치도 서귀포시 서호동 1286-1,1,1,0,0,...,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20,x,x
2,3,개방화장실,법제3조제1호,엉또폭포,제주특별자치도 서귀포시 엉또로 104,제주특별자치도 서귀포시 강정동 1561-1,1,1,0,0,...,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20,33.2672443,126.4983941
3,4,개방화장실,법제3조제1호,악근천(풍림콘도 -동쪽 바로 옆),x,제주특별자치도 서귀포시 강정동 5628 (신축),1,1,0,0,...,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20,x,x
4,5,개방화장실,법제3조제1호,베릿내,제주특별자치도 서귀포시 중문관광로 181,제주특별자치도 서귀포시 중문동 2631,1,1,0,0,...,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-20,33.2447132,126.4195338


### 위도경도 값이 없으면 API로 가져온 위도경도 값 넣어주기

In [73]:
# 데이터 확인
newJejuToilet[['WGS84위도', 'WGS84경도', 'lat', 'lng']]

,WGS84위도,WGS84경도,lat,lng
0,33.29096,126.55501,x,x
1,33.26631,126.51379,x,x
2,33.26724,126.49839,33.2672443,126.4983941
3,33.23437,126.48992,x,x
4,33.24471,126.41953,33.2447132,126.4195338
...,...,...,...,...
772,NaN,NaN,33.4863833,126.4569079
773,NaN,NaN,33.4866347,126.4375090
774,NaN,NaN,33.4750610,126.5155220
775,NaN,NaN,33.5096823,126.7156478


### na값 채우기

In [79]:
newJejuToilet['WGS84위도'].fillna(newJejuToilet['lat'], inplace=True)
newJejuToilet['WGS84경도'].fillna(newJejuToilet['lng'], inplace=True)

/var/folders/09/43kl2fx1247736ywj_tnnz500000gn/T/ipykernel_25776/767485145.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['126.9291494' '126.3447602' '126.3406787' '126.3030938' '126.3030938' 'x'
 '126.4261469' '126.3298018' '126.3071485' 'x' 'x' 'x' '126.3543658' 'x'
 'x' 'x' 'x' 'x' '126.3468168' 'x' '126.3923184' 'x' 'x' '126.3583544'
 '126.3030938' 'x' 'x' 'x' 'x' '126.7598429' 'x' 'x' '126.8514013'
 '126.8566436' '126.8620520' 'x' '126.7570207' 'x' 'x' 'x' 'x' 'x' 'x' 'x'
 '126.8804801' 'x' 'x' 'x' 'x' '126.8988005' '126.8141764' '126.8626805'
 '126.8572966' 'x' 'x' 'x' 'x' '126.6139881' '126.6139881' 'x' 'x'
 '126.6355458' '126.6740732' '126.3291907' '126.4033364' '126.8520370'
 '126.8520370' '126.8520370' '126.6341880' '126.6437192' '126.6146300'
 '126.1841306' '126.1837080' '126.1845301' '126.2960724' '126.9558798'
 '126.9530513' '126.5263272' '126.5263272' '126.5371111' '126.5268961'
 '126.5352568' '

In [81]:
newJejuToilet[['WGS84위도', 'WGS84경도']].isna().sum()

WGS84위도    0
WGS84경도    0
dtype: int64

In [83]:
# 'lat'와 'lng' 열 삭제
newJejuToilet.drop(columns=['lat', 'lng'], inplace=True)

In [87]:
newJejuToilet.tail()

,번호,구분,근거,화장실명,소재지도로명주소,소재지지번주소,남성용-대변기수,남성용-소변기수,남성용-장애인용대변기수,남성용-장애인용소변기수,...,화장실소유구분,오물처리방식,안전관리시설설치대상여부,비상벨설치여부,비상벨설치장소,화장실입구CCTV설치유무,기저귀교환대유무,기저귀교환대장소,리모델링연월,데이터기준일자
772,773,개방화장실,법제3조제13호,이호운동장,제주특별자치도 제주시 오광로6길58-11,NaN,1,2,1,0,...,공공기관-지방자치단체,수세식,Y,Y,여자화장실,N,N,없음,NaN,2023-12-31
773,774,개방화장실,법제3조제13호,외도2운동장,제주특별자치도 제주시 통물길75,NaN,2,2,1,0,...,공공기관-지방자치단체,수세식,Y,Y,여자화장실,N,N,없음,NaN,2023-12-31
774,775,개방화장실,법제3조제16호-영제3조제1항제1호,제주아트센터 외부 화장실,제주특별자치도 제주시 오남로 231,NaN,1,1,0,0,...,공공기관-지방자치단체,수세식,Y,Y,남자화장실+여자화장실,N,N,없음,NaN,2023-12-31
775,776,개방화장실,법제3조제16호-영제3조제1항제1호,동백동산(습지센타 주차장),제주특별자치도 제주시 조천읍 동백로 77,NaN,3,5,1,0,...,공공기관-지방자치단체,수세식,Y,Y,장애인화장실+남자화장실+여자화장실,N,Y,남자화장실+여자화장실,NaN,2023-12-31
776,777,개방화장실,법제3조제16호-영제3조제1항제1호,에코촌 유스호스텔 주차장,제주특별자치도 제주시 조천읍 북흘로 376-9,NaN,3,3,1,0,...,공공기관-지방자치단체,수세식,Y,Y,장애인화장실+남자화장실+여자화장실,Y,Y,여자화장실,NaN,2023-12-31


In [86]:
newJejuToilet.count(axis=1)

0      30
1      30
2      30
3      30
4      30
       ..
772    29
773    29
774    29
775    29
776    29
Length: 777, dtype: int64

### 필요한 컬럼만 뽑아서 json파일로 저장하기


In [88]:
newJejuToilet.columns

Index(['번호', '구분', '근거', '화장실명', '소재지도로명주소', '소재지지번주소', '남성용-대변기수', '남성용-소변기수',
       '남성용-장애인용대변기수', '남성용-장애인용소변기수', '남성용-어린이용대변기수', '남성용-어린이용소변기수',
       '여성용-대변기수', '여성용-장애인용대변기수', '여성용-어린이용대변기수', '관리기관명', '전화번호', '개방시간',
       '개방시간상세', '설치연월', 'WGS84위도', 'WGS84경도', '화장실소유구분', '오물처리방식',
       '안전관리시설설치대상여부', '비상벨설치여부', '비상벨설치장소', '화장실입구CCTV설치유무', '기저귀교환대유무',
       '기저귀교환대장소', '리모델링연월', '데이터기준일자'],
      dtype='object')

In [94]:
finalJejuToilet = newJejuToilet[['화장실명', '소재지도로명주소', '남성용-장애인용대변기수', '남성용-장애인용소변기수','여성용-장애인용대변기수', '개방시간상세', 'WGS84위도', 'WGS84경도', '비상벨설치여부', '비상벨설치장소', '기저귀교환대유무', '기저귀교환대장소']]

In [95]:
finalJejuToilet.to_json("Data/제주공중화장실Data_전처리.json", orient='records', force_ascii=False)